## Table of Contents
- [Preprocessing 1D **Photodiode Data** for Conversion to 2D Image Format](#Preprocessing-1D-%2A%2APhotodiode-Data%2A%2A-for-Conversion-to-2D-Image-Format)
  - [Background:](#Background%3A)
  - [Read raw photodiode data](#Read-raw-photodiode-data)
  - [Thresholding](#Thresholding)
- [🏠 Home](../../../../../welcomePage.ipynb)

# Preprocessing 1D **Photodiode Data** for Conversion to 2D Image Format

## Background:

**A method to filter raw photodiode signal is demonstrated in this section.**
The light intensity signal in the wavelength range of 500–1700 nm emitted during the building process is collected by a photodiode (C10439-11).The selected photodiode is an InGaAs type of photodiode with a Φ3 mm photosensitive area. The photodiode is installed inside the build chamber at an angle of about 45◦ facing the center of the building zone. The distance from the laser scanning area to the photodiode is approximately 250 mm. The photodiode is connected via a NI-9221 data acquisition card. The overall monitoring system is controlled by the LabView software, which provides convenience for adjusting the acquisition parameters.

<img src="./figures/1.png" alt="Drawing" style="width: 400px;" title=" Experimental setup and a schematic representation of the SLM system"/>

*Experimental setup and a schematic representation of the SLM system (The photodiode sensor in red circle)*

### Process detail

1. Thresholding. The amplitude of the photodiode signal increases rapidly when the laser starts to melt the powder bed. Meanwhile, the captured light signal before laser scanning started is close to zero and thus can be ignored. This is due to the fact that there are few light signals emitted with the wavelength range of 500–1700 nm before the laser scanning starts. Hence, the raw photodiode signal was just analyzed by thresholding to extract the photodiode data during the building process. In this work, the photodiode signals during the laser melting process were extracted by removing the photodiode signals that were less than the amplitude of 2. The threshold value of 2 was selected to ensure all the photodiode signals during laser building process are extracted.
2. The raw 1D acoustic or photodiode signal with a length of $M^2$ is extracted through a running window, where $M^2$ is the size of theconverted image. The running window is selected by considering that it can not only increase the number of training samples but also help train the quality monitoring models with local sensor information. Then, the running window with $M^2$ samples in the time domain fulfills the pixels of the image by sequence. Detailly, consecutive 1D signal samples with a length of $M$ fulfill rows of the converted image in sequence. For the $L(i), i = 1, 2, …, M^2$ sample in the 1D signal, its coordinate corresponding to the pixel in the image pixel is $P(j,k)$.

$P(j, k)=\operatorname{round}\left\{\frac{L((j-1) \times M+k)-\min (L)}{\max (L)-\min (L)} \times 255\right\},(j, k=1,2, \ldots, M)$

### The information users need to provide:
1. The raw photodiode data local directory path.
2. The converted image data save folder local directory path.
3. The number of sample.
4. The length of each image.
5. The sliding step size

## Read raw photodiode data

Provide the raw photodiode data local directory path in the following text box, the raw data will be loaded.

(*Example*: ../data/photodiode)

#### Press <font color = '#2195F2'>**Play**</font> to Select Raw Photodiode Data to Load

In [ ]:
import os
from scipy.io import loadmat
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display

# Create text input widget for path input
path_input = widgets.Text(
    description='Directory Path:',
    placeholder='Paste or type the directory path here',
    style={'description_width': 'initial'}
)

# Create a button to confirm the directory selection
confirm_button = widgets.Button(description="Confirm Path")

text = widgets.Text(value="../data/photodiode", description="Example")

# Variable to hold the path
# Default path: 
# /Users/zyichao/Desktop/phd/Codes_Additive_Manufacturing/Two_Sensor_Fusion_JMP/Data/photodiode
global directory_path
directory_path = ""

data_dict = {}

# Define button click event handler
def on_confirm_button_clicked(b):
    global directory_path
    directory_path = path_input.value
    print(f"Path selected: {directory_path}")
    for filename in os.listdir(directory_path):
        if filename.endswith('.mat'):
            file_path = os.path.join(directory_path, filename)
            mat_data = loadmat(file_path)
            
            data_name = os.path.splitext(filename)[0]
            print(data_name+' is loaded')
            data_dict[data_name] = mat_data


    
confirm_button.on_click(on_confirm_button_clicked)

# Display widgets
display(text)
display(path_input, confirm_button)


#### Press <font color = '#2195F2'>**Play**</font> to Display the Selected Raw Data Sample

In [ ]:
x = list(data_dict.keys())[0]
y = list(data_dict[x].keys())[3]

plt.figure(figsize=(8,5),dpi=100)
plt.plot(data_dict[x][y])
plt.title("Raw photodiode signal sample")
plt.show()

## Thresholding

Provide the image data **save folder directory path**. **The number of sample**, **length of each image**, **sliding step size**, and **the number of the image** can be modified to preview the output of the image data. Once all the variables are determined, the user can convert the signal data to image data.

(*Example*: ../data/image_data/photodiode/)


### The 2-D photodiode image data modification and visualization

#### Press <font color = '#2195F2'>**Play**</font> to Select the Save Folder

In [ ]:
# Create text input widget for path input
path_input = widgets.Text(
    description='Svae Forlder Directory Path:',
    placeholder='Paste or type the directory path here',
    style={'description_width': 'initial'}
)

# Create a button to confirm the directory selection
confirm_button = widgets.Button(description="Confirm Path")
text = widgets.Text(value="../data/image_data/photodiode/", description="Example")

# Variable to hold the path
# Default path: 
# /Users/zyichao/Desktop/phd/Codes_Additive_Manufacturing/Two_Sensor_Fusion_JMP/Data/photodiode
global save_folder
save_folder = ""

# Define button click event handler
def on_confirm_button_clicked(b):
    global save_folder
    save_folder = path_input.value
    print(f"Path selected: {directory_path}")
            
confirm_button.on_click(on_confirm_button_clicked)

# Display widgets
display(text)
display(path_input, confirm_button)

#### Press <font color = '#2195F2'>**Play**</font> to Modify the Preprocessing Properties

In [ ]:
def photodiode(a,b,c,d):
    ## a:file position, b:size of each image, c:Sliding step size, d:the number of the image
    
    filename = list(data_dict.keys())[a]
    parts = filename.split('_')
    phd_grade = int(parts[1]) 
    phd_num = int(parts[2])
    
    filtered_data = []
    data_all = []
    for m in range(1,18):
        x = data_dict[filename]['cDAQ1Mod4ai'+str(m)]
        y = x[x>2]
        filtered_data.append(y) 

    for i in range(1, 18):  # based on each layer, 17 layers in total
        data = filtered_data[i-1]
        N = len(data)
        W = b * b  # Window size  
        # c: Sliding step size, middle value 10 * 10000 // 520 = 192, range from 50 to 300
        hdcs = (N - W) // c + 1  # Number of sliding steps 

        for k in range(1, hdcs + 1):
            one_data = data[(k-1) * c: (k-1) * c + W]
            # Normalize and scale the data to 0-255
            one_data = np.round((one_data - np.min(one_data)) / (np.max(one_data) - np.min(one_data)) * 255).astype(np.uint8)
            # Reshape the image
            one_data = one_data.reshape((b, b)) 
            
            data_all.append(one_data)


    result = filename
    print(f"The selected sample name: {result}")
    return plt.imshow(data_all[d],cmap='gray')

interact(photodiode,
         a=widgets.IntSlider(min=0, max=len(data_dict.keys())-1, step=1, value=0, description='The number of sample:',style={'description_width': 'initial'}),
         b=widgets.IntSlider(min=32, max=128, step=1, value=0, description='Length of each image:',style={'description_width': 'initial'}),
         c=widgets.IntSlider(min=50, max=300, step=1, value=0, description='Sliding step size:',style={'description_width': 'initial'}),
         d=widgets.IntSlider(min=0, max=100, step=1, value=0, description='The number of the image:',style={'description_width': 'initial'}),
)

### Save the modified image data

#### Press <font color = '#2195F2'>**Play**</font> to Save the Modified Image Data

In [ ]:
for i in range(3):
    folder_path = os.path.join(save_folder, str(i))
    try:
        os.makedirs(folder_path)
        print("Directory ", folder_path, " Created ")
    except FileExistsError:
        print("Directory ", folder_path, " already exists")

for j in range(len(data_dict.keys())):
    
    filename = list(data_dict.keys())[j]

    parts = filename.split('_')
    phd_grade = int(parts[1]) 
    phd_num = int(parts[2])
    
    filtered_data = []
    data_all = []
    for m in range(1,18):
        x = data_dict[filename]['cDAQ1Mod4ai'+str(m)]
        y = x[x>2]
        filtered_data.append(y)

    folder = save_folder + str(phd_grade)
    n = 32  # Size of each image

    for i in range(1, 18):  # based on each layer, 17 layers in total
        data = filtered_data[i-1]
        N = len(data)
        W = n * n  # Window size
        hd = 10 * 10000 // 520  # Sliding step size
        hdcs = (N - W) // hd + 1  # Number of sliding steps

        for k in range(1, hdcs + 1):
            one_data = data[(k-1) * hd: (k-1) * hd + W]
            # Normalize and scale the data to 0-255
            one_data = np.round((one_data - np.min(one_data)) / (np.max(one_data) - np.min(one_data)) * 255).astype(np.uint8)
            # Reshape the image
            one_data = one_data.reshape((n, n)) 
            
            data_all.append(one_data)
            # Save the image
            image_filename = f'p_{phd_num}_{i}_{k}.png'
            image_path = os.path.join(folder, image_filename)
            Image.fromarray(one_data).save(image_path)

    print(filename, "Processing complete.")

### <center>[🏠 Home](../../../../../welcomePage.ipynb)</center>